In [1]:
import numpy as np
import gc
import pandas as pd
import torch
from torch import nn
from torch_geometric.nn import GCNConv, SAGEConv, GAE
from torch_geometric.data import Data
from tqdm import tqdm
import polars as pl
from collections import defaultdict
from sklearn.preprocessing import StandardScaler
import os
import sys
sys.path.append('../models')
from utils import *
from models import *


device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

c:\Users\wikid\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cpu


In [4]:
# use features extracted by Node2Vec technique that uses Biase Random Walk to extract node embeddings
node2vec_embeddings  = np.load("../output/node2vec_embeddings.npy")

In [5]:
node2vec_embeddings

array([[ 0.2377169 ,  0.09706575, -0.33888143, ..., -0.41141638,
        -0.02868662,  0.05329121],
       [ 0.07449519,  0.31996906,  0.59136367, ...,  0.41820285,
         0.07786004,  0.16922267],
       [-0.3861353 , -0.2294153 ,  0.23723997, ..., -0.5580095 ,
        -0.12203991, -0.0035588 ],
       ...,
       [-0.22501837,  0.3269598 ,  0.7061448 , ...,  0.6103053 ,
        -0.50257504, -0.09074241],
       [-1.0086778 , -0.24329227, -0.24240543, ...,  0.12725711,
        -0.18508181, -0.45989668],
       [-0.16124317, -0.01643375,  0.10715707, ...,  0.11199374,
        -0.02729834, -0.32067934]], dtype=float32)